In [ ]:
import ipywidgets as widgets
from IPython.display import display

from agage_archive.config import Paths, data_file_list
from agage_archive.widgets import update_instrument_site, plot_to_output, show_netcdf_info

In [ ]:
network = "agage"
paths = Paths(network, errors="ignore_inputs")

# Find available files and plot

Execute the following cell and then select options from the menus that appear.

In [ ]:
# Species names from the output directory structure
species = sorted(set([s.split("/")[1] for s in data_file_list(network, paths.output_path, errors="ignore_inputs")[2]]))

In [ ]:
# Create dropdown widget
species_dropdown = widgets.Dropdown(
    options=species,
    description='Species:',
    disabled=False,
    default=species[0]
)

# Selection widget for network and site
instrument_site = widgets.SelectMultiple(
    options=update_instrument_site({"new": species[0]}, network, None),
    description='Site, instrument:',
    disabled=False,
    indent=True
)

# Plotting button
plot_button = widgets.Button(description="Plot")

# Output widget
output = widgets.Output()
output_netcdf = widgets.Output()

# Update network and site dropdown when species is changed
species_dropdown.observe(lambda change:
                        update_instrument_site(change, network, instrument_site),
                        names="value")

# Plot to output when button is clicked
plot_button.on_click(lambda x: plot_to_output(x, network, 
                                              species_dropdown.value,
                                              instrument_site.value,
                                              output))
plot_button.on_click(lambda x: show_netcdf_info(x, network,
                                                species_dropdown.value,
                                                instrument_site.value,
                                                output_netcdf))                                         

display(species_dropdown)
display(instrument_site)
display(plot_button)
display(output)
display(output_netcdf)